In [2]:
import json
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime, timedelta, timezone

In [3]:
!rm config.json  # remove if any
!wget https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.json # get the config file

rm: cannot remove 'config.json': No such file or directory
--2025-07-10 18:38:36--  https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 708 [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]     708  --.-KB/s    in 0s      

2025-07-10 18:38:36 (7.47 MB/s) - ‘config.json’ saved [708/708]



In [16]:
# Create a function that loads the congif

def load_config(path="config.json"):
    with open(path, "r") as f:
        return json.load(f)

In [17]:
#  Create a function that sends the email notification

def send_email(subject, body):
    config = load_config()
    email_conf = config["email"]

    msg = MIMEMultipart()
    msg["From"] = email_conf["sender_email"]
    msg["To"] = email_conf["receiver_email"]
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    with smtplib.SMTP(email_conf["smtp_server"], email_conf["smtp_port"]) as server:
        server.starttls()
        server.login(email_conf["sender_email"], email_conf["sender_password"])
        server.send_message(msg)

    print("Email is sent successfully.")

In [18]:
#  Create a function that sends the telegram notification

def send_telegram(message):
  config = load_config()
  token = config["telegram"]["bot_token"]
  chat_id = config["telegram"]["chat_id"]

  url = f"https://api.telegram.org/bot{token}/sendMessage"
  payload = {
      "chat_id" : chat_id,
      "text": message
  }

  response = requests.post(url, data= payload)

  if response.status_code == 200:
    print("Message is sent successfully.")
  else:
    print(f"Error: {response.status_code} - {response.text}")


In [20]:
# Combine notification functions together

def notification(subject, message):
  try:
    send_email(subject, message)
  except Exception as excep:
    print(f"Email Error: {excep}")

  try:
    send_telegram(message)
  except Exception as excep:
    print(f"Telegram Error: {excep}")

In [21]:
# Create a function that gets the remote jobs

def remote_jobs():
    url = "https://remoteok.com/api"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        jobs = response.json()
        return jobs[1:]
    else:
        print(f"Error: {response.status_code}")
        return []

In [22]:
def relevant_job(job, config):
    title = job.get("position", "").lower()
    job_date_str = job.get("date", "")


    if not any(kw.lower() in title for kw in config["search_keywords"]):
        return False

    try:
        job_time = datetime.strptime(job_date_str, "%Y-%m-%dT%H:%M:%S%z")
        if datetime.now(timezone.utc) - job_time > timedelta(hours=24):
            return False
    except Exception as e:
        print(f"Error: {e}")
        return False

    return True

In [23]:
jobs = remote_jobs()
config = load_config()

for job in jobs:
    if relevant_job(job, config):
        message = f" {job['position']} @ {job.get('company', 'Unknown')}\n {job.get('location', 'Remote')}\n🔗 {job.get('url', '')}"
        notification("New Job Post!", message)